This project is intended to serve as practice for cleaning and analysis with the python pandas module. The data has been scraped from the classified section of the German ebay website, Kleinazeigen. The data can be downloaded from the link below
https://data.world/data-society/used-cars-data

Data Dictionary:

dateCrawled - When this ad was first crawled. All field-values are taken from this date

name - Name of the car

seller - Whether the seller is private or a dealer

offerType - The type of listing

price - The price on the ad to sell the car

abtest - Whether the listing is included in an A/B test

vehicleType - The vehicle Type

yearOfRegistration - The year in which the car was first registered

gearbox - The transmission type

powerPS - The power of the car in PS

model - The car model name

kilometer - How many kilometers the car has driven

monthOfRegistration - The month in which the car was first registered

fuelType - What type of fuel the car uses

brand - The brand of the car

notRepairedDamage - If the car has a damage which is not yet repaired

dateCreated - The date on which the eBay listing was created

nrOfPictures - The number of pictures in the ad

postalCode - The postal code for the location of the vehicle

lastSeenOnline - When the crawler saw this ad last online


In [3]:
import pandas as pd
import numpy as np
import math

In [4]:
engine='python'
autos = pd.read_csv('autos.csv', encoding='Latin-1', header=0)

In [5]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

In [6]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Will need to convert german words 
vehicleType,gearbox, model, fuelType, and notRepairedDamage have Null values

In [7]:
for i in autos.columns:
    print(i)

dateCrawled
name
seller
offerType
price
abtest
vehicleType
yearOfRegistration
gearbox
powerPS
model
odometer
monthOfRegistration
fuelType
brand
notRepairedDamage
dateCreated
nrOfPictures
postalCode
lastSeen


# Rename columns and covert data types and ID questionable data

In [8]:
auto_columns = autos.columns.copy()

In [9]:
auto_columns = [
                'date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
                'vehicle_type', 'registration_year', 'gearbox', 'power_PS', 'model',
                'odometer', 'registration_month', 'fuel_type', 'brand',
                'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
                'last_seen'
               ]

In [10]:
autos.columns = auto_columns

In [11]:
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Changed to cammel case and reworded headers

In [12]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-10 15:36:24,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


seller, offer_type, abtest, unrepaired_damage, and gearbox columns have only two unique values each.
rd.


In [13]:
autos['offer_type'].unique()

array(['Angebot', 'Gesuch'], dtype=object)

In [14]:
autos.loc[autos['offer_type'] == 'Angebot', 'offer_type'].shape

(49999,)

In [15]:
autos['seller'].unique()

array(['privat', 'gewerblich'], dtype=object)

In [16]:
autos.loc[autos['seller']=='privat', 'seller'].shape

(49999,)

In [17]:
autos['gearbox'].unique()

array(['manuell', 'automatik', nan], dtype=object)

In [18]:
autos.loc[autos['gearbox']=='manuell', 'gearbox'].shape

(36993,)

Seller and offer type columns have all the same value except one entry and can be dropped.

In [19]:
autos['registration_year'].unique()

array([2004, 1997, 2009, 2007, 2003, 2006, 1995, 1998, 2000, 2017, 2010,
       1999, 1982, 1990, 2015, 2014, 1996, 1992, 2005, 2002, 2012, 2011,
       2008, 1985, 2016, 1994, 1986, 2001, 2018, 2013, 1972, 1993, 1988,
       1989, 1967, 1973, 1956, 1976, 4500, 1987, 1991, 1983, 1960, 1969,
       1950, 1978, 1980, 1984, 1963, 1977, 1961, 1968, 1934, 1965, 1971,
       1966, 1979, 1981, 1970, 1974, 1910, 1975, 5000, 4100, 2019, 1959,
       9996, 9999, 6200, 1964, 1958, 1800, 1948, 1931, 1943, 9000, 1941,
       1962, 1927, 1937, 1929, 1000, 1957, 1952, 1111, 1955, 1939, 8888,
       1954, 1938, 2800, 5911, 1500, 1953, 1951, 4800, 1001])

Some of the registration years cannot be correct. Date is too old

In [20]:
autos.dtypes

date_crawled          object
name                  object
seller                object
offer_type            object
price                 object
abtest                object
vehicle_type          object
registration_year      int64
gearbox               object
power_PS               int64
model                 object
odometer              object
registration_month     int64
fuel_type             object
brand                 object
unrepaired_damage     object
ad_created            object
nr_of_pictures         int64
postal_code            int64
last_seen             object
dtype: object

Convert Price, model unrepaired damage to floats

In [21]:
print(autos['price'].unique().shape)

(2357,)


In [22]:
autos['price'] = autos['price'].str.replace('$', "")\
                               .str.replace("\'", "")\
                               .str.replace(",", "")\
                               .astype(float)

In [23]:
autos['odometer'].unique()

array(['150,000km', '70,000km', '50,000km', '80,000km', '10,000km',
       '30,000km', '125,000km', '90,000km', '20,000km', '60,000km',
       '5,000km', '100,000km', '40,000km'], dtype=object)

In [24]:
autos['odometer'] = autos['odometer']\
                    .str.replace("km","")\
                    .str.replace("\''", "")\
                    .str.replace(",", "")\
                    .astype(int)

In [25]:
autos['odometer'].unique()

array([150000,  70000,  50000,  80000,  10000,  30000, 125000,  90000,
        20000,  60000,   5000, 100000,  40000])

In [26]:
autos.rename(columns = {'odometer':'odometer_km'}, inplace=True)

In [27]:
autos['odometer_km'].describe()

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [28]:
autos['odometer_km'].value_counts().sort_index(ascending=False)

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
40000       819
30000       789
20000       784
10000       264
5000        967
Name: odometer_km, dtype: int64

In [29]:
autos['price'].unique().shape

(2357,)

In [30]:
autos['price'].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

In [31]:
autos['price'].describe().apply(lambda x: format(x, 'f'))

count       50000.000000
mean         9840.043760
std        481104.380500
min             0.000000
25%          1100.000000
50%          2950.000000
75%          7200.000000
max      99999999.000000
Name: price, dtype: object

remove cares with zero price. Also 1 billion looks unreasonable

In [32]:
autos = autos[autos['price'].between(1,1000000)]

In [33]:
autos.shape

(48568, 20)

In [34]:
autos['price'].describe().apply(lambda x: format(x, 'f'))

count     48568.000000
mean       5950.340656
std       11963.134750
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      999999.000000
Name: price, dtype: object

In [35]:
autos[autos['price']>500000]

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
514,2016-03-17 09:53:08,Ford_Focus_Turnier_1.6_16V_Style,privat,Angebot,999999.0,test,kombi,2009,manuell,101,focus,125000,4,benzin,ford,nein,2016-03-17 00:00:00,0,12205,2016-04-06 07:17:35
37585,2016-03-29 11:38:54,Volkswagen_Jetta_GT,privat,Angebot,999990.0,test,limousine,1985,manuell,111,jetta,150000,12,benzin,volkswagen,ja,2016-03-29 00:00:00,0,50997,2016-03-29 11:38:54
43049,2016-03-21 19:53:52,2_VW_Busse_T3,privat,Angebot,999999.0,test,bus,1981,manuell,70,transporter,150000,1,benzin,volkswagen,NaN,2016-03-21 00:00:00,0,99880,2016-03-28 17:18:28


Some cars have a list price of 999,999 which is unreasonable. 
Ford focus and VW can't sell for tha much.

In [36]:
autos[autos['price']>200000]

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
514,2016-03-17 09:53:08,Ford_Focus_Turnier_1.6_16V_Style,privat,Angebot,999999.0,test,kombi,2009,manuell,101,focus,125000,4,benzin,ford,nein,2016-03-17 00:00:00,0,12205,2016-04-06 07:17:35
12682,2016-03-28 22:48:01,Porsche_GT3_RS__PCCB__Lift___grosser_Exklusiv_...,privat,Angebot,265000.0,control,coupe,2016,automatik,500,911,5000,3,benzin,porsche,nein,2016-03-28 00:00:00,0,70193,2016-04-05 03:44:51
14715,2016-03-30 08:37:24,Rolls_Royce_Phantom_Drophead_Coupe,privat,Angebot,345000.0,control,cabrio,2012,automatik,460,NaN,20000,8,benzin,sonstige_autos,nein,2016-03-30 00:00:00,0,73525,2016-04-07 00:16:26
34723,2016-03-23 16:37:29,Porsche_Porsche_911/930_Turbo_3.0__deutsche_Au...,privat,Angebot,299000.0,test,coupe,1977,manuell,260,911,100000,7,benzin,porsche,nein,2016-03-23 00:00:00,0,61462,2016-04-06 16:44:50
35923,2016-04-03 07:56:23,Porsche_911_Targa_Exclusive_Edition__1_von_15_...,privat,Angebot,295000.0,test,cabrio,2015,automatik,400,911,5000,6,benzin,porsche,nein,2016-04-03 00:00:00,0,74078,2016-04-03 08:56:20
36818,2016-03-27 18:37:37,Porsche_991,privat,Angebot,350000.0,control,coupe,2016,manuell,500,911,5000,3,benzin,porsche,nein,2016-03-27 00:00:00,0,70499,2016-03-27 18:37:37
37585,2016-03-29 11:38:54,Volkswagen_Jetta_GT,privat,Angebot,999990.0,test,limousine,1985,manuell,111,jetta,150000,12,benzin,volkswagen,ja,2016-03-29 00:00:00,0,50997,2016-03-29 11:38:54
37840,2016-03-21 10:50:12,Porsche_997,privat,Angebot,220000.0,test,coupe,2008,manuell,415,911,30000,7,benzin,porsche,nein,2016-03-21 00:00:00,0,69198,2016-04-06 04:46:14
38299,2016-03-28 22:25:25,Glas_BMW_mit_Wasser,privat,Angebot,250000.0,test,NaN,2015,NaN,0,x_reihe,5000,0,NaN,bmw,NaN,2016-03-28 00:00:00,0,60489,2016-03-28 22:25:25
43049,2016-03-21 19:53:52,2_VW_Busse_T3,privat,Angebot,999999.0,test,bus,1981,manuell,70,transporter,150000,1,benzin,volkswagen,NaN,2016-03-21 00:00:00,0,99880,2016-03-28 17:18:28


Ok, Rolls royce, porch, BMW could get 2-300,000. Remove the other three cars

In [37]:
autos.iloc[514,:]

date_crawled                                        2016-03-26 22:55:10
name                  Ideales_Anfaenger_auto__fuer_nach_bestandener_...
seller                                                           privat
offer_type                                                      Angebot
price                                                               333
abtest                                                             test
vehicle_type                                                 kleinwagen
registration_year                                                  1995
gearbox                                                         manuell
power_PS                                                             33
model                                                             corsa
odometer_km                                                      150000
registration_month                                                    9
fuel_type                                                       

In [38]:
autos = autos[autos['price'].between(1,400000)]

In [39]:
autos.shape

(48565, 20)

In [40]:
autos['price'].describe().apply(lambda x: format(x, 'f'))

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: object

In [41]:
autos['price'].value_counts().sort_index(ascending=False).head()

350000.0    1
345000.0    1
299000.0    1
295000.0    1
265000.0    1
Name: price, dtype: int64

Now lets look at the date columns. Should I convert to datetime object?

In [42]:
autos[['date_crawled', 'ad_created', 'last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


In [43]:
autos['date_crawled'].str[:10].unique()

array(['2016-03-26', '2016-04-04', '2016-03-12', '2016-04-01',
       '2016-03-21', '2016-03-20', '2016-03-16', '2016-03-22',
       '2016-03-15', '2016-03-31', '2016-03-23', '2016-03-29',
       '2016-03-17', '2016-03-05', '2016-03-06', '2016-03-28',
       '2016-03-10', '2016-04-03', '2016-03-19', '2016-04-02',
       '2016-03-14', '2016-04-05', '2016-03-11', '2016-03-07',
       '2016-03-08', '2016-03-27', '2016-03-09', '2016-03-25',
       '2016-03-18', '2016-03-30', '2016-03-24', '2016-03-13',
       '2016-04-06', '2016-04-07'], dtype=object)

In [44]:
autos['date_crawled'].str[:10].unique().shape

(34,)

In [45]:
autos['date_crawled'].unique().shape

(46882,)

There are 34 different dates all occuring between march and april. They are represented as strings. Convert to datetime objects. I don't think the exact time will be very useful so I will remove this

In [46]:
import datetime
autos['date_crawled'] = autos['date_crawled']\
                        .str[:10]\
                        .apply(lambda x: datetime.date.fromisoformat(x))

In [47]:
autos['date_crawled'].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

Distribution of dates crawled is pretty uniform. Probably occured at roughly the same times every day with a few exceptions

In [48]:
autos['ad_created'].unique()

array(['2016-03-26 00:00:00', '2016-04-04 00:00:00',
       '2016-03-12 00:00:00', '2016-04-01 00:00:00',
       '2016-03-21 00:00:00', '2016-03-20 00:00:00',
       '2016-03-16 00:00:00', '2016-03-22 00:00:00',
       '2016-03-14 00:00:00', '2016-03-31 00:00:00',
       '2016-03-23 00:00:00', '2016-03-29 00:00:00',
       '2016-03-17 00:00:00', '2016-03-05 00:00:00',
       '2016-03-06 00:00:00', '2016-03-28 00:00:00',
       '2016-03-10 00:00:00', '2016-04-03 00:00:00',
       '2016-03-19 00:00:00', '2016-04-02 00:00:00',
       '2016-03-15 00:00:00', '2016-04-05 00:00:00',
       '2016-03-11 00:00:00', '2016-03-07 00:00:00',
       '2016-03-08 00:00:00', '2016-03-27 00:00:00',
       '2016-03-09 00:00:00', '2016-03-25 00:00:00',
       '2016-03-18 00:00:00', '2016-03-30 00:00:00',
       '2016-03-24 00:00:00', '2016-03-13 00:00:00',
       '2016-04-06 00:00:00', '2016-03-04 00:00:00',
       '2016-04-07 00:00:00', '2016-02-24 00:00:00',
       '2016-03-02 00:00:00', '2016-02-29 00:0

Looks like we can drop the time part of the time stamp

In [49]:
autos['ad_created'] = autos['ad_created']\
                        .str[:10]\
                        .apply(lambda x: datetime.date.fromisoformat(x))

In [50]:
pd.options.display.max_rows = 80
print(autos['ad_created'].value_counts(normalize=True, dropna=False).sort_index())

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
2016-02-22    0.000021
2016-02-23    0.000082
2016-02-24    0.000041
2016-02-25    0.000062
2016-02-26    0.000041
2016-02-27    0.000124
2016-02-28    0.000206
2016-02-29    0.000165
2016-03-01    0.000103
2016-03-02    0.000103
2016-03-03    0.000865
2016-03-04    0.001483
2016-03-05    0.022897
2016-03-06 

Adds created appear to start off slow than ramp up in March and 
slow down again in June.

In [51]:
autos['last_seen'].unique().shape

(38474,)

In [52]:
autos[autos['last_seen'].isnull() == True].shape

(0, 20)

There are no null values

In [53]:
autos['last_seen'] = autos['last_seen']\
                        .str[:10]\
                        .apply(lambda x: datetime.date.fromisoformat(x))

In [54]:
print(autos['last_seen'].value_counts(normalize=True, dropna=False).sort_index())

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64


Distribution looks pretty constant throughout March and April

In [55]:
autos['registration_year'].describe(include='all')

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

In [56]:
pd.options.display.max_rows = 95
autos['registration_year'].value_counts().sort_index()

1000       1
1001       1
1111       1
1800       2
1910       5
1927       1
1929       1
1931       1
1934       2
1937       4
1938       1
1939       1
1941       2
1943       1
1948       1
1950       3
1951       2
1952       1
1953       1
1954       2
1955       2
1956       4
1957       2
1958       4
1959       6
1960      23
1961       6
1962       4
1963       8
1964      12
1965      17
1966      22
1967      26
1968      26
1969      19
1970      38
1971      26
1972      33
1973      23
1974      24
1975      18
1976      21
1977      22
1978      44
1979      34
1980      85
1981      28
1982      41
1983      51
1984      51
1985      95
1986      72
1987      72
1988     135
1989     174
1990     347
1991     339
1992     370
1993     425
1994     629
1995    1227
1996    1373
1997    1951
1998    2363
1999    2897
2000    3156
2001    2636
2002    2486
2003    2699
2004    2703
2005    2936
2006    2670
2007    2277
2008    2215
2009    2085
2010    1589
2011    1623

There are some errors in dates as the min registration year is 1000 and the max is 9999

In [57]:
print(autos[(autos['registration_year'] < 1900) | (autos['registration_year'] > 2016)].shape,
      autos.shape)


(1884, 20) (48565, 20)


We will loose 1884 rows from a total of 48,565 which is ~ 4% of the data
I think it is justified

In [58]:
autos = autos[autos['registration_year'].between(1900, 2016)]

In [59]:
autos.shape

(46681, 20)

In [60]:
autos['registration_year'].value_counts(normalize=True).sort_index()

1910    0.000107
1927    0.000021
1929    0.000021
1931    0.000021
1934    0.000043
1937    0.000086
1938    0.000021
1939    0.000021
1941    0.000043
1943    0.000021
1948    0.000021
1950    0.000064
1951    0.000043
1952    0.000021
1953    0.000021
1954    0.000043
1955    0.000043
1956    0.000086
1957    0.000043
1958    0.000086
1959    0.000129
1960    0.000493
1961    0.000129
1962    0.000086
1963    0.000171
1964    0.000257
1965    0.000364
1966    0.000471
1967    0.000557
1968    0.000557
1969    0.000407
1970    0.000814
1971    0.000557
1972    0.000707
1973    0.000493
1974    0.000514
1975    0.000386
1976    0.000450
1977    0.000471
1978    0.000943
1979    0.000728
1980    0.001821
1981    0.000600
1982    0.000878
1983    0.001093
1984    0.001093
1985    0.002035
1986    0.001542
1987    0.001542
1988    0.002892
1989    0.003727
1990    0.007433
1991    0.007262
1992    0.007926
1993    0.009104
1994    0.013474
1995    0.026285
1996    0.029412
1997    0.0417

Car registration peaks around 1996-2009. most cars registered before this date 
are probably too old to put on sale and there are fewer very recent cars 
going on sale.

# Investigate top 10% of the most frequent cars

In [61]:
autos['brand'].value_counts(normalize=True).sort_values()

lada              0.000578
lancia            0.001071
rover             0.001328
trabant           0.001392
daewoo            0.001500
jaguar            0.001564
saab              0.001649
land_rover        0.002099
subaru            0.002142
jeep              0.002271
daihatsu          0.002506
dacia             0.002635
chrysler          0.003513
chevrolet         0.005698
suzuki            0.005934
porsche           0.006127
alfa_romeo        0.006641
kia               0.007069
honda             0.007840
mitsubishi        0.008226
mini              0.008762
volvo             0.009147
sonstige_autos    0.009811
hyundai           0.010025
toyota            0.012703
citroen           0.014010
smart             0.014160
mazda             0.015188
nissan            0.015274
skoda             0.016409
seat              0.018273
fiat              0.025642
peugeot           0.029841
renault           0.047150
ford              0.069900
audi              0.086566
mercedes_benz     0.096463
o

Aggregate over the top ten percent

In [62]:
market = autos['brand'].value_counts(normalize=True).sort_values() 
tenpct = market[market >= 0.01]

In [63]:
tenpct

hyundai          0.010025
toyota           0.012703
citroen          0.014010
smart            0.014160
mazda            0.015188
nissan           0.015274
skoda            0.016409
seat             0.018273
fiat             0.025642
peugeot          0.029841
renault          0.047150
ford             0.069900
audi             0.086566
mercedes_benz    0.096463
opel             0.107581
bmw              0.110045
volkswagen       0.211264
Name: brand, dtype: float64

In [64]:
for i in tenpct.index:
    print(i)

hyundai
toyota
citroen
smart
mazda
nissan
skoda
seat
fiat
peugeot
renault
ford
audi
mercedes_benz
opel
bmw
volkswagen


In [65]:
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_PS', 'model',
       'odometer_km', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

In [66]:
topautos = {}
for brand in tenpct.index:
    selected_rows = autos[autos["brand"]==brand]
    meanprice = selected_rows['price'].mean()
    topautos[brand] = meanprice

In [67]:
#make an ordered dict
for k in sorted(topautos, key=topautos.get, reverse=True):
    print(k+'\t'+"{:.2f}".format(topautos[k]))
    
    

audi	9336.69
mercedes_benz	8628.45
bmw	8332.82
skoda	6368.00
volkswagen	5402.41
hyundai	5365.25
toyota	5167.09
nissan	4743.40
seat	4397.23
mazda	4112.60
citroen	3779.14
ford	3749.47
smart	3580.22
peugeot	3094.02
opel	2975.24
fiat	2813.75
renault	2474.86


Audi, mercedis_benz and bmw top the list. Makes siense. Smaller cars are
at the bottom. skoda looked funny but it became subsidiary of volksagen 
in 1991. I guess it makes sense that german cars top the list since the data comes from a german website. Followed by Japanese cars and then Italian cars. Only one US car in Ford.

In [68]:
def series_from_data(df, series, agg_col, new_col):
    d = {}
    for i in series.index:
        selected_rows = df[df["brand"]==i]
        mean = selected_rows[agg_col].mean()
        d[i] = mean
    d_series = pd.Series(d)
    #df = pd.DataFrame(d_series, columns=[new_col])

    return d_series

In [69]:
mean_mi_series = series_from_data(autos, tenpct, 'odometer_km', 'Mean Mileage' )
mean_price_series = series_from_data(autos, tenpct, 'price', 'Mean Price' )

In [70]:
df = pd.DataFrame(mean_price_series, columns=['Mean Price'])

In [71]:
df['Mean Mileage'] = mean_mi_series

In [72]:
pd.options.display.float_format = '{:,.2f}'.format

In [73]:
df.sort_values(by=['Mean Price', 'Mean Mileage'], ascending=False)

,Mean Price,Mean Mileage
audi,"9,336.69","129,157.39"
mercedes_benz,"8,628.45","130,788.36"
bmw,"8,332.82","132,572.51"
skoda,"6,368.00","110,848.56"
volkswagen,"5,402.41","128,707.16"
hyundai,"5,365.25","106,442.31"
toyota,"5,167.09","115,944.35"
nissan,"4,743.40","118,331.00"
seat,"4,397.23","121,131.30"
mazda,"4,112.60","124,464.03"


It looks like Audi Mercedes Benz and bmw don't depreciate much with increased mileage. They have the highest mileage coupled to the highest prices.

# Replace German words with English and drop non-useful columns

In [74]:
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_PS', 'model',
       'odometer_km', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

In [75]:
autos['seller'].unique()

array(['privat', 'gewerblich'], dtype=object)

In [76]:
seller_d = {'privat':'private', 'gewerblich':'commercial'}
autos['seller'] = autos['seller'].map(seller_d)

In [77]:
autos['seller'].unique()

array(['private', 'commercial'], dtype=object)

In [78]:
autos['offer_type'].unique()

array(['Angebot'], dtype=object)

Not providing useful information so drop this column

In [79]:
autos = autos.drop('offer_type', axis=1)

In [80]:
autos['vehicle_type'].unique()


array(['bus', 'limousine', 'kleinwagen', 'kombi', nan, 'coupe', 'suv',
       'cabrio', 'andere'], dtype=object)

In [81]:
d_vehicle_type = {'bus':'bus', 'limousine':'limousine', 'coupe':'coupe',\
                  'suv':'suv','kleinwagen':'compact', 'kombi':'station wagon',\
                 'cabrio':'convertible','andere':'other'}
autos['vehicle_type'] = autos['vehicle_type'].map(d_vehicle_type)

In [82]:
autos['vehicle_type'].unique()

array(['bus', 'limousine', 'compact', 'station wagon', nan, 'coupe',
       'suv', 'convertible', 'other'], dtype=object)

In [83]:
autos['unrepaired_damage'].unique()

array(['nein', nan, 'ja'], dtype=object)

In [84]:
d_damage = {'nein':'no', 'ja':'yes'}
autos['unrepaired_damage'] = autos['unrepaired_damage'].map(d_damage)

In [85]:
autos['unrepaired_damage'].unique()

array(['no', nan, 'yes'], dtype=object)

# Most Common brand/model combinations 



In [86]:
gp = autos.groupby('brand')['model'].value_counts().sort_values(ascending=False)[0:10]

In [87]:
gp

brand          model   
volkswagen     golf        3707
bmw            3er         2615
volkswagen     polo        1609
opel           corsa       1592
volkswagen     passat      1349
opel           astra       1348
audi           a4          1231
mercedes_benz  c_klasse    1136
bmw            5er         1132
mercedes_benz  e_klasse     958
Name: model, dtype: int64

All of the top brands are German owned. Not surprising coming from a German used car website. Volkswagen takes most of the top brands. Volkswagen AG also owns Audi. Opel was owned by GM from 1929-2017. Mercedes benz and bmw are also popular German cars. It would be intresting to see the distribution of years for these cars.

In [88]:
for i,j in gp.index:
    print(i,j)
    

volkswagen golf
bmw 3er
volkswagen polo
opel corsa
volkswagen passat
opel astra
audi a4
mercedes_benz c_klasse
bmw 5er
mercedes_benz e_klasse


In [89]:
pd.options.display.max_rows = 200
brand_model_dict = {}
for brand, model in gp.index:
    selected_rows = autos[(autos["brand"]==brand) & (autos["model"] == model)]
    registration_years = selected_rows['registration_year'].value_counts()
    brand_model_dict[(brand,model)] = registration_years
#print(brand_model_dict.items())
df_bm = pd.DataFrame(brand_model_dict)
df_bm[df_bm.isnull() == True] = 0
df_bm = df_bm.astype(int)

In [90]:
df_bm

,volkswagen,bmw,volkswagen,opel,volkswagen,opel,audi,mercedes_benz,bmw,mercedes_benz
,golf,3er,polo,corsa,passat,astra,a4,c_klasse,5er,e_klasse
1910,0,0,0,1,0,0,0,0,0,0
1973,0,1,0,0,0,0,0,0,0,0
1977,0,1,1,0,0,0,0,0,1,0
1978,0,1,0,0,0,0,0,0,0,1
1979,2,1,0,0,0,0,0,0,0,0
1980,3,0,1,0,1,0,0,0,1,2
1981,2,1,0,0,0,0,0,0,0,1
1982,2,2,0,0,0,0,0,0,0,2
1983,5,2,0,0,1,0,0,0,2,1


In [91]:
df_bm[['volkswagen', 'bmw', 'opel', 'audi', 'mercedes_benz']]

volkswagen              bmw       opel       audi mercedes_benz         
           golf polo passat  3er  5er corsa astra   a4      c_klasse e_klasse
1910          0    0      0    0    0     1     0    0             0        0
1973          0    0      0    1    0     0     0    0             0        0
1977          0    1      0    1    1     0     0    0             0        0
1978          0    0      0    1    0     0     0    0             0        1
1979          2    0      0    1    0     0     0    0             0        0
1980          3    1      1    0    1     0     0    0             0        2
1981          2    0      0    1    0     0     0    0             0        1
1982          2    0      0    2    0     0     0    0             0        2
1983          5    0      1    2    2     0     0    0             0        1
1984         10    1      0    1    0     0     0    0             0        2
1985          7    1      1    4    1     0     0    0             0        3
1986         13    4      1    2    2     0     0    0             0        1
1987         12    1      6    7    1     1     0    0             0        4
1988         20    4      1   10    2     1     0    0             0        6
1989         22    2      6   10    8     0     1    0             0       12
1990         70   22      2   24    9     4     1    0             0       14
1991         62   24      6   22    3     1     2    0             2        9
1992         47   18      9   26    6     6     5    0             0       16
1993         86   17     11   39    7     8    16    0             8       17
1994         93   42     24   70   10    22    35    5            34       15
1995        189   91     37  114   18    73    54   55            36       23
1996        157  138     35  121   34    91    53   65            34       32
1997        167  131     82  144   67   157    61   68            52       48
1998        269   90    109  153   78   121   104   43            57       41
1999        251  100     75  176   59   131   117   50            59       56
2000        233  126     61  181   63   172   108   54            72       57
2001        152   98     78  183   57   125    73   57           105       45
2002        137   86     53  186   49    97    72   89            90       53
2003        125   93     78  176   69    96    61   78            81       86
2004        134   53     66  138  116    61    62   95            74       80
2005        165   57     93  132  105    59    80   83            54       55
2006        160   59    114  132   68    40    90   86            46       38
2007        125   54     85  144   47    55    65   93            52       35
2008        110   38     80  102   62    52    65   87            70       27
2009        170   41     52   70   39    59    49   66            50       38
2010        142   46     32   59   50    21    30   36            28       52
2011        115   39     57   63   53    24    50   44            49       39
2012        125   35     34   31   20    24    32   30            33       19
2013         80   22     18   21    7     8     7   13            17       13
2014         65    7      9   13    7    11    12   10            16        3
2015         56    5      6    9    1     3     2    8             3        0
2016        124   62     26   43   10    68    41   16            14       12

Both VW golf and BMW 3 peak earlyier (around 1988) compared to other cars. The earlier models may be more popular. VW Golf seems to keep up it's popularity the longes continuing strong into 2016 while others have tailed of (In particular the Mercedes E-class)

# Average Price Vs. Mileage

In [92]:
autos.groupby('odometer_km')['price'].mean()

odometer_km
5000      8,873.52
10000    20,550.87
20000    18,448.48
30000    16,608.84
40000    15,499.57
50000    13,812.17
60000    12,385.00
70000    10,927.18
80000     9,721.95
90000     8,465.03
100000    8,132.70
125000    6,214.02
150000    3,767.93
Name: price, dtype: float64

The average price drops dsharply with increasing mileage

# Comparison of cars with and without damage

In [93]:
pd.options.display.max_rows = 200
top_brands = df.index.to_list()
top_autos = autos[autos['brand'].isin(top_brands)]
damage = top_autos.groupby(['brand', 'unrepaired_damage'])['price'].mean()
#df_damage = pd.DataFrame(damage).reset_index(level=[0])
damage

brand          unrepaired_damage
audi           no                  10,914.96
               yes                  3,324.68
bmw            no                   9,437.71
               yes                  3,512.64
citroen        no                   4,289.71
               yes                  1,944.54
fiat           no                   3,446.07
               yes                  1,146.89
ford           no                   4,660.33
               yes                  1,375.49
hyundai        no                   6,194.59
               yes                  2,417.63
mazda          no                   5,110.77
               yes                  1,418.24
mercedes_benz  no                   9,798.40
               yes                  3,921.82
nissan         no                   5,765.23
               yes                  1,947.41
opel           no                   3,660.53
               yes                  1,367.03
peugeot        no                   3,674.02
               yes    

In [94]:
df_damage = damage.to_frame().reset_index([1])

In [95]:
df_damage

,unrepaired_damage,price
brand,,
audi,no,"10,914.96"
audi,yes,"3,324.68"
bmw,no,"9,437.71"
bmw,yes,"3,512.64"
citroen,no,"4,289.71"
citroen,yes,"1,944.54"
fiat,no,"3,446.07"
fiat,yes,"1,146.89"
ford,no,"4,660.33"


It looks like damaged cars sell for 1/2 to 1/3 the price of undamaged cars of the same brand. However, Toyota appears to hold it's value better than other brands. There could also be a bigger difference for cars that have older registration years.